# CYBER Master

This project is to process the data of recorded voices and baseline measurements for the attachment style interaction with depression and anxiety, in a sense this will work a the ETL or pre-processing of the data.

This notebook has a single purpose expressed in the following steps:
- Gather the answers from the survey DB
- Process the answers by uniques and present a report of the demographics, attachment styles and HADS answers
- From each answer, get the recordings from blob storage.
- For each recording transcribe them and associate them to the answer
- Store the resulting data in the DB for further analysis

In [14]:
# -*- coding: utf-8 -*-

# Import the necessary libraries
import os
import sys
import json
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
import subprocess

load_dotenv(find_dotenv())

CONSENT_SURVEY_ID = "8"
STUDY_SURVEY_ID = "7"


def transcribe_audio(file_path, language="en"):
    # Use the local vosk via CLI to get the transcription
    result_file = file_path.replace(".wav", ".txt")
    subprocess.run(
        [
            "vosk-transcriber",
            "-i",
            file_path,
            "-l",
            language,
            "-o",
            result_file,
            "--log-level",
            "ERROR",
        ]
    )
    # Read the transcription
    with open(result_file, "r") as transcription_file:
        transcription = transcription_file.read()
        return transcription

## Connect to the DB

In [15]:
import mysql.connector

connection = mysql.connector.connect(
    user=os.getenv("DB_USERNAME"),
    password=os.getenv("DB_PASSWORD"),
    host=os.getenv("DB_HOST"),
    database=os.getenv("DB_NAME"),
    port=os.getenv("DB_PORT"),
)
cursor = connection.cursor()

# query the answers table
query = (
    f"SELECT * FROM surveys_answer where surveys_answer.survey_id = {STUDY_SURVEY_ID}"
)
cursor.execute(query)
answers = cursor.fetchall()

In [16]:
# Parse the answers
json_answers = []

for answer in answers:
    dict_answer = json.loads(json.loads(answer[1]))
    if dict_answer.get("response-uuid") not in [answer.get("response-uuid") for answer in json_answers]:
        json_answers.append({"id": answer[0], **json.loads(json.loads(answer[1]))})
full_answers = [answer for answer in json_answers if len(answer.keys()) == 62]
# get answers by language in a dictionary
answers_by_language = {}
answers_by_language = {
    "en": len([answer for answer in full_answers if answer["language"] == "en"]),
    "es": len([answer for answer in full_answers if answer["language"] == "es"]),
    "pt": len([answer for answer in full_answers if answer["language"] == "pt"]),
}
print(answers_by_language)

{'en': 0, 'es': 7, 'pt': 0}


In [17]:
# Get the recording info from the recording table
recordings_query = f"SELECT * FROM surveys_recording"  # where surveys_recording.answer_id in ({','.join([str(answer.get('id')) for answer in full_answers])})"
cursor.execute(recordings_query)
recordings = cursor.fetchall()
print(recordings[0])

(131, 'recordings/blob', None, 'en', '88bd9dcedc08411a800024e83806387c')


In [18]:
for answer in full_answers:
    for recording in recordings:
        for key, value in answer.items():
            if not isinstance(value, str) or len(value) < 4:
                continue
            if value.replace("-", "") != recording[-1]:
                continue
            answer[key] = f"{recording[1].replace('recordings/', '')}.wav"


print(full_answers[0])

{'id': 119, 'language': 'es', 'response-uuid': '4788e176-f802-4c27-9124-61541ac9ce6a', 'Demographic - Age': '31', 'Demographic - Gender': 'female', 'Demographic - Education level': 'bachelor', 'Demographic - Relationship status': 'single', 'Demographic - Native speaker': 'yes', 'Demographic - speech diagnosis': 'no', 'Demographic - psychiatric disorder': 'no', 'Demographic - Substances': 'no', 'RQ1': 'A', 'RQ2': '6', 'RQ3': '1', 'RQ4': '5', 'RQ5': '2', 'HADS 1': '1', 'HADS 2': '1', 'HADS 3': '0', 'HADS 4': '1', 'HADS 5': '0', 'HADS 6': '0', 'HADS 7': '2', 'HADS 8': '0', 'HADS 9': '1', 'HADS 10': '2', 'HADS 11': '3', 'HADS 12': '1', 'HADS 13': '2', 'HADS 14': '1', 'Image 1 - 1': 'blob_nrnej2h.wav', 'Image 1 - 2': 'blob_6Pu57EX.wav', 'Image 1 - 3': 'blob_DSHsnaO.wav', 'Image 1 - 4': 'blob_tLTvyIY.wav', 'Image 2 - 1': 'blob_oQmoc6K.wav', 'Image 2 - 2': 'blob_wVigQOC.wav', 'Image 2 - 3': 'blob_81DcWTq.wav', 'Image 2 - 4': 'blob_yiLAcqi.wav', 'Image 3 - 1': 'blob_2ZzcIQ2.wav', 'Image 3 - 2'

In [19]:
print(full_answers[-1])

{'id': 332, 'language': 'es', 'response-uuid': '67b712a1-0883-4b37-8a27-046070165fb1', 'Demographic - Age': '35', 'Demographic - Gender': 'male', 'Demographic - Education level': 'master', 'Demographic - Relationship status': 'relationship', 'Demographic - Native speaker': 'yes', 'Demographic - speech diagnosis': 'no', 'Demographic - psychiatric disorder': 'no', 'Demographic - Substances': 'no', 'RQ1': 'A', 'RQ2': '7', 'RQ3': '6', 'RQ4': '3', 'RQ5': '2', 'HADS 1': '1', 'HADS 2': '0', 'HADS 3': '1', 'HADS 4': '2', 'HADS 5': '0', 'HADS 6': '0', 'HADS 7': '2', 'HADS 8': '0', 'HADS 9': '0', 'HADS 10': '1', 'HADS 11': '2', 'HADS 12': '0', 'HADS 13': '3', 'HADS 14': '3', 'Image 1 - 1': 'blob_KEVjp5s.wav', 'Image 1 - 2': 'blob_tdN7bxu.wav', 'Image 1 - 3': 'blob_F4eR8SJ.wav', 'Image 1 - 4': 'blob_FviMkbb.wav', 'Image 2 - 1': 'blob_R94PBYr.wav', 'Image 2 - 2': 'blob_LKyMZxD.wav', 'Image 2 - 3': 'blob_19OS2U5.wav', 'Image 2 - 4': 'blob_E11VnWI.wav', 'Image 3 - 1': 'blob_zKyEAmW.wav', 'Image 3 - 

## Getting the recordings per answer

In [20]:
from azure.storage.blob import BlobServiceClient

CONTAINER_NAME = "recordings"

blob_service = BlobServiceClient.from_connection_string(
    os.getenv("AZURE_CONNECTION_STRING")
)
local_path = "./recordings"
# create the folder if not alrady
if not os.path.exists(local_path):
    os.mkdir(local_path)

# Sanity check, listing the blobs in the container
container_client = blob_service.get_container_client(CONTAINER_NAME)
blob_list = []
for blob in container_client.list_blobs():
    if "blob" in blob.name:
        blob_list.append(blob.name)

In [21]:
# Download the blobs into the recordings folder if not already downloaded
for blob in blob_list:
    result_file_name = blob.split("/")[-1]
    result_file_name = f"{result_file_name}.wav"
    if result_file_name not in os.listdir(local_path):
        with open(file=f"{local_path}/{result_file_name}", mode="wb") as audio_file:
            download_stream = container_client.download_blob(blob)
            audio_file.write(download_stream.readall())

# list all the files in the folder
files = os.listdir(local_path)
print(len(files))

525


## Vosk

We are using [vosk](https://alphacephei.com/vosk/) to transcribe the audio into text, then assign it to the answer for further processing

In [24]:
for answer in full_answers:
    for key, value in answer.items():
        if not isinstance(value, str) or not value.endswith(".wav"):
            continue
        if value not in files:
            print(f"File {value} not found")
            continue
        if value.replace(".wav", ".txt") in os.listdir(local_path):
            print(f"File {value} already transcribed")
            answer[key] = open(f"{local_path}/{value.replace('.wav', '.txt')}", "r").read()
        answer[key] = transcribe_audio(f"{local_path}/{value}", language=answer["language"])

File blob_nrnej2h.wav already transcribed


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=11 max-active=4000 lattice-beam=4
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from /home/dvelez/.cache/vosk/vosk-model-small-es-0.42/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from /home/dvelez/.cache/vosk/vosk-model-small-es-0.42/graph/HCLr.fst /home/dvelez/.cache/vosk/vosk-model-small-es-0.42/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winfo /home/dvelez/.cache/vosk/vosk-model-small-es-0.42/graph/phones/word_boundary.int


File blob_6Pu57EX.wav already transcribed


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=11 max-active=4000 lattice-beam=4
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from /home/dvelez/.cache/vosk/vosk-model-small-es-0.42/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from /home/dvelez/.cache/vosk/vosk-model-small-es-0.42/graph/HCLr.fst /home/dvelez/.cache/vosk/vosk-model-small-es-0.42/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winfo /home/dvelez/.cache/vosk/vosk-model-small-es-0.42/graph/phones/word_boundary.int


File blob_DSHsnaO.wav already transcribed


...

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=11 max-active=4000 lattice-beam=4
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from /home/dvelez/.cache/vosk/vosk-model-small-es-0.42/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from /home/dvelez/.cache/vosk/vosk-model-small-es-0.42/graph/HCLr.fst /home/dvelez/.cache/vosk/vosk-model-small-es-0.42/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winfo /home/dvelez/.cache/vosk/vosk-model-small-es-0.42/graph/phones/word_boundary.int


File blob_lNNGj9P.wav already transcribed


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=11 max-active=4000 lattice-beam=4
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from /home/dvelez/.cache/vosk/vosk-model-small-es-0.42/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from /home/dvelez/.cache/vosk/vosk-model-small-es-0.42/graph/HCLr.fst /home/dvelez/.cache/vosk/vosk-model-small-es-0.42/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winfo /home/dvelez/.cache/vosk/vosk-model-small-es-0.42/graph/phones/word_boundary.int


File blob_1CsfM8n.wav already transcribed


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=11 max-active=4000 lattice-beam=4
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from /home/dvelez/.cache/vosk/vosk-model-small-es-0.42/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from /home/dvelez/.cache/vosk/vosk-model-small-es-0.42/graph/HCLr.fst /home/dvelez/.cache/vosk/vosk-model-small-es-0.42/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winfo /home/dvelez/.cache/vosk/vosk-model-small-es-0.42/graph/phones/word_boundary.int


In [23]:
# save the answers in a json lines file
now_date = datetime.now().strftime("%Y-%m-%d")
with open(f"answers-{now_date}.json", "w") as answers_file:
    for answer in full_answers:
        answers_file.write(json.dumps(answer))
        answers_file.write("\n")